In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets

In [2]:
def create_target(t):
        target_vector = np.zeros(10)
        for i in range(10):
            if i == t:
                target_vector[i] = 1
        return target_vector
    
def hsig(x):
    z = np.clip(x, 0, 1)
    return np.copy(z)

def d_hsig(x):
    z = (x > 0) & (x < 1)
    return np.copy(z)

def relu(x):
    y = x * (x > 0)
    return np.copy(y)

def softmax(y):
    e_y = np.exp(y - np.max(y))
    return e_y / e_y.sum()

In [3]:
digits = datasets.load_digits()
data = digits.data
targets = digits.target

inputs = data - np.mean(data)
inputs = inputs/(np.std(data))

In [42]:
n_x = 64
n_h = 50
n_y = 10

epsilon = 0.5

W1 = np.random.normal(0, (4/(n_x + n_y)), (n_x, n_h))
W2 = np.random.normal(0, (4/(n_h)), (n_h, n_y))

bh = np.random.normal(0, 4/(n_x + n_y), n_h)
by = np.random.normal(0, 4/(n_h), n_y)

alpha1 = 0.01
alpha2 = 0.005
beta = 1

h = np.random.uniform(0, 1, n_h)
y = np.random.uniform(0, 1, n_y)

for ex in range(1000):
    rnd = np.random.randint(0, 1497)
    x = inputs[rnd]
    t = create_target(targets[rnd])
    h = np.random.uniform(0, 1, n_h)
    y = np.random.uniform(0, 1, n_y)
    
    #Free Phase
    for itr in range(100):
        dh = d_hsig(h) * (np.dot(x, W1) + np.dot(y, W2.T) + bh) - h
        dy = d_hsig(y) * (np.dot(h, W2) + by) - y
        
        h = hsig(h + epsilon * dh)
        y = hsig(y + epsilon * dy)
        
    h_free = np.copy(h)
    y_free = np.copy(y)
    
    #Weakly Clamped Phase
    for itr in range(20):
        dy = d_hsig(y) * (np.dot(h, W2) + by) - y + beta * (t - y)
        dh = d_hsig(h) * (np.dot(x, W1) + np.dot(y, W2.T) + bh) - h
        
        h = hsig(h + epsilon * dh)
        y = hsig(y + epsilon * dy)
        
    h_clamped = np.copy(h)
    y_clamped = np.copy(y)
    
    W1 += alpha1 * (1/beta) * (np.outer(x, h_clamped) - np.outer(x, h_free))
    W2 += alpha2 * (1/beta) * (np.outer(h_clamped, y_clamped) - np.outer(h_free, y_free))
    
    if ex % 10 == 0:
        print(np.dot(t - y_free, t - y_free))

1.62769071392
1.11480244602
1.16124647839
1.33972846747
1.0735621617
0.82864902133
0.963971114513
0.540219144205
1.09042856799
0.896733306166
1.18386711836
1.04809901558
1.10347504989
0.831893015491
0.833882851683
0.889432374854
0.463479448109
0.642271529629
1.16034606976
1.25850147048
1.16458642457
0.459092195787
0.411284558763
0.371695367572
0.759274546458
0.397033038264
0.59896654838
0.683720892617
1.0561256894
0.480435562814
0.691925478948
0.328467506493
0.434185807014
0.555343799155
1.17026172913
0.222282563578
0.199406429124
0.76190331359
0.799733630751
1.10711842971
0.57399358125
0.465365025766
1.12004248612
0.775241459992
0.532932084847
1.06475772835
0.49170096044
0.418124687372
0.263923089792
0.277351425953
0.419428329759
0.531846913644
1.08353574294
0.232298554432
0.647526226923
1.18131439026
0.308966252172
0.483964457267
0.329595067585
0.117174202362
1.15259683521
0.942653489294
0.105729643131
0.356512020611
0.146814408922
1.07434990266
0.606581847793
0.357983962101
0.181023

In [43]:
score = 0
for test in range(100):
    rnd = np.random.randint(1497, 1797)
    x = inputs[rnd]
    t = create_target(targets[rnd])
    h = np.random.uniform(0, 1, n_h)
    y = np.random.uniform(0, 1, n_y)
    
    #Free Phase
    for itr in range(100):
        dh = d_hsig(h) * (np.dot(x, W1) + np.dot(y, W2.T) + bh) - h
        dy = d_hsig(y) * (np.dot(h, W2) + by) - y
        
        h = hsig(h + epsilon * dh)
        y = hsig(y + epsilon * dy)
        
    h_free = np.copy(h)
    y_free = np.copy(y)
    
    if np.argmax(y_free) == targets[rnd]:
        score += 1
    
print(score/100)

0.7
